### Bogdanov-Takens bifurcation
The Andronov-Hopf and saddle-node bifurcation sets can intersect each other, and since the Wilson-Cowan model can have more than one equilibrium, the intersections could correspond to simultaneous bifurcations of different equilibria. 

When both bifurcations happen to be of the same equilibrium the bifurcation is called a Bogdanov- Takens bifurcation, and the Jacobian matrix $J$ verifies,

\begin{equation}
    \text{det} (J) = 0,\quad \text{Tr} (J) = 0 
\end{equation}

this is the condition that we will use to find Bogdanov-Takens bifurcation

### Saddle Node on Invariant Cycle

Let's understand first saddle node on limit cycles. Considering a dynamical system having a saddle-node bifurcation. 

![image](\figures\saddle-node_bifurcation.png)

If the initial condition $x(0)$ starts from the right-hand side of the saddle (open circle), then $x(t)$ increases and leaves a small neighborhood containing the saddle and the node. To study x(t) far away from the neighborhood requires global knowledge of the function f(x, A) . Depending on the function, x(t) can go to infinity or be attracted to another stable equilibrium, limit cycle, chaotic attractor, etc. 

It is also possible that x(t) can return to the neighborhood. The simplest case for this occurs when the left and the right branches of the invariant manifold M make a loop.

![image](\figures\saddle-node_limit_cycle.png)

When x(O) is near the right branch of M, it leaves a neighborhood of the saddle, makes a rotation along M, and approaches the node. Such an excursion in neuroscience applications is called an action potential or a spike. Obviously, the node is an attractor for all x inside the limit cycle and could be an attractor for all points outside the cycle. The activity x(t) of a dynamical system having phase portrait as in Figure 2.29a or b always converges to the node, at least for x(O) close enough to the limit cycle. The invariant manifold M is itself an orbit, which starts from the saddle (open circle in Figure 2.29a) and ends in the node (shaded circle) . Such an orbit is said to be heteroclinic.

### Code

As for the programing we start by defining the same things as the previous task.

In [ ]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt


def S(x):
    """
    S function
    """
    return np.tanh(x/2)/2

def S_prime(x):
    return 1/4*(1-np.tanh(x/2)**2)

#Define the System of equations
def wilson_covan (y, wEE, wEI, wIE, wII, P, t):
    E, I = y
    P_E, P_I = P
    tau_E, tau_I = t
    dEdt = (-E + S (wEE * E - wEI * I + P_E)) / tau_E
    dIdt = (-I + S (wIE * E - wII * I + P_I)) / tau_I
    return np.array([dEdt, dIdt])

#Define the Jacobian matrix 
def jacobian(y, wEE, wEI, wIE, wII, P, t):
    E, I = y
    P_E, P_I = P
    tau_E, tau_I = t

    dEdE = (-1 + wEE * S_prime(wEE * E - wEI * I + P_E))/tau_E
    dEdI = -wEI * S_prime(wEE * E - wEI * I +P_E) / tau_E
    dIdE = wIE * S_prime(wIE * E - wII * I + P_I) / tau_I
    dIdI = (-1 - wII*S_prime(wIE * E - wII * I + P_I))/ tau_I

    return np.array([dEdE, dEdI, dIdE, dIdI])

#### Find the fixed points and classify the stability

The same as before the same code is used to find the fixed points.

In [ ]:
def fixed_points(wEE, wEI, wIE, wII, tol, initial_guess, P = ([0, 0]), t=[1, 1]):
    fixed_points = [[0, 0]]
    for guess in initial_guess:
        sol = optimize.root(wilson_covan, guess, args=(wEE, wEI, wIE, wII, P, t))
        if sol.success:
            if not any(np.allclose(sol.x, fp, atol=tol) for fp in fixed_points):
                fixed_points.append(sol.x)
    return np.array(fixed_points)

# Stability analysis (Jacobian method)
def stability_analysis(wEE, wIE, wEI, wII, fixed_points):
    stable = []
    for point in fixed_points:
        J = jacobian(point, wEE, wEI, wIE, wII, P=[0, 0], t=[1, 1])
        new_J = np.array([[J[0], J[1]], [J[2], J[3]]])
        eigenvalues = np.linalg.eigvals(new_J)
        if np.all(np.real(eigenvalues) < 0):
            stable.append("Stable")
        elif np.any(np.real(eigenvalues) > 0):
            stable.append("Unstable")
    return np.array(stable)

#### Prepare inital values and other parameters.

In [ ]:
# Phase diagram
wEI, wII = 10, 1  # Fixed inhibitory parameters
wEE_values = np.linspace(0.1, 30, 200)
wIE_values = np.linspace(0.1, 20, 200)
stability_map = np.zeros((len(wEE_values), len(wIE_values)))

tol_EI = 1e-5 #The toletance in E and I, to consider if 2 close fixed points are the same.

N = 100
x = 2
initial_guesses = np.zeros((N, 2))

for i in range(N):
    initial_guesses[i] = np.array([np.random.uniform(-x, x), np.random.uniform(-x, x)])

#### Classification of bifurcations
